# Checking the functions of the pipeline

Imports

In [19]:
from src.unify_format import unify_data_format
from src.variable_matching import match_variables
from src.temporal_resolution import consolidate_time_resolution
from src.match import aggregate_and_merge_files

import os
import xarray as xr

from src.config import standard_col_names

Downloading data

In [2]:
file_path_MetO = "/Users/bean/Documents/Doctorate/Fall_24/Mokbel8735_Adv_Databases/climate_data_integration/data/raw/HadISST.2.2.0.0_sea_ice_concentration.nc"
met = xr.open_dataset(file_path_MetO)

file_path_NOAA = "/Users/bean/Documents/Doctorate/Fall_24/Mokbel8735_Adv_Databases/climate_data_integration/data/raw/icec.mon.mean.nc"
noa = xr.open_dataset(file_path_NOAA)

### Standardizing variable names
**match_variables**

Current file column names:

In [49]:
met.dims.keys()

/var/folders/mr/2yqg72xx2l3bscg4fdjn78_c0000gn/T/ipykernel_5304/2489301570.py:1: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  met.dims.keys()


KeysView(FrozenMappingWarningOnValuesAccess({'time': 2048, 'nv': 2, 'latitude': 180, 'longitude': 360}))

In [58]:
met.data_vars.keys()

KeysView(Data variables:
    time_bnds  (time, nv) float32 16kB ...
    sic        (time, latitude, longitude) float32 531MB ...)

In [57]:
noa.dims.keys()

/var/folders/mr/2yqg72xx2l3bscg4fdjn78_c0000gn/T/ipykernel_5304/2353949293.py:1: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  noa.dims.keys()


KeysView(FrozenMappingWarningOnValuesAccess({'lat': 180, 'lon': 360, 'time': 2040}))

In [59]:
noa.data_vars.keys()

KeysView(Data variables:
    icec     (time, lat, lon) float32 529MB ...)

In [20]:
st = match_variables(input_data=[file_path_MetO, file_path_NOAA], standard_col_names=standard_col_names)

/Users/bean/Documents/Doctorate/Fall_24/Mokbel8735_Adv_Databases/climate_data_integration/src/variable_matching.py:27: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  for current_name in data.dims.keys():  # Check dimension names
/Users/bean/Documents/Doctorate/Fall_24/Mokbel8735_Adv_Databases/climate_data_integration/src/variable_matching.py:27: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  for current_name in data.dims.keys():  # Check dimension names


In [21]:
new_met = xr.open_dataset(st[0])
print(new_met.dims.keys())
print(new_met.data_vars.keys())
new_noa = xr.open_dataset(st[1])
print(new_noa.dims.keys())
print(new_noa.data_vars.keys())

KeysView(FrozenMappingWarningOnValuesAccess({'datetime': 2048, 'sea_ice_temp': 2, 'latitude': 180, 'longitude': 360}))
KeysView(Data variables:
    time_bnds  (datetime, sea_ice_temp) float32 16kB ...)
KeysView(FrozenMappingWarningOnValuesAccess({'latitude': 180, 'longitude': 360, 'datetime': 2040}))
KeysView(Data variables:
    sea_ice_temp  (datetime, latitude, longitude) float32 529MB ...)


/var/folders/mr/2yqg72xx2l3bscg4fdjn78_c0000gn/T/ipykernel_7510/1962799246.py:2: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(new_met.dims.keys())
/var/folders/mr/2yqg72xx2l3bscg4fdjn78_c0000gn/T/ipykernel_7510/1962799246.py:5: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(new_noa.dims.keys())


### Get same file format and check the grid map information
**unify_data_format**

### Resample datasets into monthly resolution
**consolidate_time_resolution**

### Aggregating and merging files
**aggregate_and_merge_files**